In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

**Iniciando analise**

In [ ]:
#importando algumas bibliotecas
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from pylab import rcParams

%matplotlib inline

In [ ]:
# lendo o arquivo
df = pd.read_csv('/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')

**Exploracao dos dados**

In [ ]:
# verificando as colunas
list(df.columns)

In [ ]:
# verificando a dimencao do dataframe
df.shape

Temos 7043 observacoes e 21 colunas

In [ ]:
# Visualizacao inicial do dataframe
df.head(10).T

**Nota: ** Primeiro de tudo, vamos excuir custumerID porque nao ira afetar a probabilidade de rotacao dos clientes.

In [ ]:
df.drop(['customerID'], axis=1, inplace=True)

In [ ]:
# dataframe sem a coluna costomerID
df.head(10).T

**Verificando a variavel a qual queremos prever**

In [ ]:
df.Churn.value_counts()

In [ ]:
# Realizando a representacao grafica da distribuicao das classes da variavel a qual desejamos prever
label = df['Churn'].value_counts(sort = True).index
sizes = df['Churn'].value_counts(sort = True)

colors = ['Whitesmoke', 'red']
explode = (0.1,0)

rcParams['figure.figsize'] = 8,8

plt.pie(sizes, explode=explode, labels=label, colors=colors,
       autopct='%1.1f%%', shadow=True, startangle=270,)

plt.title('Porcentagem de rotatividade entre os clientes')
plt.show()

Como podemos ver aqui, as classes dessa variavel estao desbalanceadas, precisaremos antes de realizar a predicao tomar alguma atitude quanto a isso.

**Missing values**<br/>
Modelos de machine learning normalmente nao trabalham bem com missing values, por este motivo, vamos verificar a existencia deste valores.

In [ ]:
df.info()

In [ ]:
total = df.isnull().sum().sort_values(ascending=False) # pegando a quantidade total de valores missing
percent = df.isnull().sum() / df.isnull().count().sort_values(ascending=False) # Dividindo a quantidade de valores missings pela quantidade de observacoes.

missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Porcentagem']) # concatenando o total com a porcentagem
display(missing_data)

Como podemos ver, aparentemente nenhum valor missing, contudo, bastante valores do tipo object, seja pelo fato do pandas nao ter reconhecido o real tipo do dado ou seja por que ele esta no tipo texto, vamos alterar o tipo deste dados para o tipo numerico, pois os modelos de machine learning em geral nao trabalham muito bem com dados categoricos.

**Realizando o preprocessamento dos dados.**

Churn: Vamos comecar pela variavel target, transformando a em 0 quando o valor for 'No' e 1 quando o valor for 'Yes'.

In [ ]:
df.Churn = df.Churn.map(lambda s: 1 if s == 'Yes' else 0)

Contract: vamos realizar o processo de one-hot encode nesta variavel.

In [ ]:
#verificando os possiveis valores da variavel Contract
df.Contract.value_counts()

In [ ]:
df = pd.get_dummies(data=df, columns=['Contract'])

Dependents: Transformar o valores 'Yes' em 1 e 'No' em 0

In [ ]:
df.Dependents.value_counts()

In [ ]:
df.Dependents = df.Dependents.map(lambda s: 1 if s == 'Yes' else 0)

DevicePrtotection: Neste caso aqui, consideramos o valor No internet service como No e atribuiremos 0 para para a variavel.

In [ ]:
df.DeviceProtection.value_counts()

In [ ]:
df.DeviceProtection = df.DeviceProtection.map(lambda s: 1 if s == 'Yes' else 0)

InternetService: A partir dessa coluna criaremos outras tres informando se a pessoa possui ou nao internet e outras informando se o cliente possui conexao DSL ou FIBRA OPTICA

In [ ]:
df.InternetService.value_counts()

In [ ]:
df.TemInternet = df.InternetService.map(lambda s: 0 if s == 'No' else 1)
df.DSL = df.InternetService.map(lambda s: 1 if s == 'DSL' else 0)
df.FibraOptica = df.InternetService.map(lambda s: 1 if s == 'Fiber optic' else 0)

In [ ]:
# como as informacoes desta coluna ja esta em outras colunas ela nao nos tera mais serventia.
df.drop(['InternetService'], axis=1, inplace=True)

MonthlyCharges: Esta coluna ja esta no formato numerico, a principio nao precisaremos mecher nela.

In [ ]:
df.MonthlyCharges.head(5)

MultipleLines: Como ja possuimos ja possuimos uma coluna que informa se o cliente possui ou nao o servico de telefone pegaremos apenas a informacao se ele tem ou nao multiplas linhas.

In [ ]:
df.MultipleLines.value_counts()

In [ ]:
df.MultipleLines = df.MultipleLines.map(lambda s: 1 if s == 'Yes' else 0)

OnlineBackup: Caso o cliente possua o servico de backup online atribuiremos 1 a variavel caso contrario 0

In [ ]:
df.OnlineBackup.value_counts()

In [ ]:
df.OnlineBackup = df.OnlineBackup.map(lambda s: 1 if s == 'Yes' else 0)

OnlineSecurity: Caso o cliente possua o servico de seguranca online atribuiremos 1 a variavel caso contrario 0

In [ ]:
df.OnlineSecurity.value_counts()

In [ ]:
df.OnlineSecurity = df.OnlineSecurity.map(lambda s: 1 if s == 'Yes' else 0)

PaperlessBilling: 1 para 'Yes' 0 para 'No'

In [ ]:
df.PaperlessBilling.value_counts()

In [ ]:
df.PaperlessBilling = df.PaperlessBilling.map(lambda s: 1 if s == 'Yes' else 0)

Partner: 1 para 'Yes' 0 para 'No'

In [ ]:
df.Partner.value_counts()

In [ ]:
df.Partner = df.Partner.map(lambda s: 1 if s == 'Yes' else 0)

PaymentMethod: Utilizaremos a tecnica de One-Hot encode nesta variavel.

In [ ]:
df.PaymentMethod.value_counts()

In [ ]:
df = pd.get_dummies(data=df, columns=['PaymentMethod'])

PhoneService: 1 para 'Yes' 0 para 'No'

In [ ]:
df.PhoneService.value_counts()

In [ ]:
df.PhoneService = df.PhoneService.map(lambda s: 1 if s == 'Yes' else 0)

SeniorCitizen: 1 para 'Yes' 0 para 'No'

In [ ]:
df.SeniorCitizen.value_counts()

In [ ]:
df.SeniorCitizen = df.SeniorCitizen.map(lambda s: 1 if s == 'Yes' else 0)

StreamingMovies: 1 para 'Yes' caso contrario 0

In [ ]:
df.StreamingMovies.value_counts()

In [ ]:
df.StreamingMovies = df.StreamingMovies.map(lambda s: 1 if s == 'Yes' else 0)

StreamingTV: 1 para 'Yes' caso contrario 0

In [ ]:
df.StreamingTV.value_counts()

In [ ]:
df.StreamingTV = df.StreamingTV.map(lambda s: 1 if s == 'Yes' else 0)

TechSupport: 1 para 'Yes' caso contrario 0

In [ ]:
df.TechSupport.value_counts()

In [ ]:
df.TechSupport = df.TechSupport.map(lambda s: 1 if s == 'Yes' else 0)

TotalCharges: Esta coluna ja esta no formato numerico, contudo, se reparar bem existem 11 observacoes que nao sao numeros, possivelmente algum espaco em branco.

In [ ]:
df.TotalCharges.value_counts()

In [ ]:
(df.TotalCharges == ' ').sum()

Ha 11 observacoes com este valor em branco, optei por descarta estas obsevacoes.

In [ ]:
df = df[df.TotalCharges != ' ']

In [ ]:
#convertendo a coluna para numerica
df.TotalCharges = pd.to_numeric(df.TotalCharges)

gender: Utilizarei o one-hot encoce nesta variavel.

In [ ]:
df.gender.value_counts()

In [ ]:
df = pd.get_dummies(data=df, columns=['gender'])

Tenure:

In [ ]:
df.tenure.head()

Vamos verificar agora como esta a cara do nosso data frame

In [ ]:
df.info()

In [ ]:
df.head().T

In [ ]:
df.shape

Agora que os dados estao devidamente tratados, vamos comecar a mexer com o modelo.

Importando as bibliotecas que iremos utilizar no modelo

In [ ]:
pip install scikit-learn==0.22.1

In [ ]:
#pip install -U imbalanced-learn

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
#from imblearn.under_sampling import NearMiss

Separando as variaveis preditoras da variavel que sera predita.


In [ ]:
X = df.drop(['Churn'], axis=1)
y = df.Churn

In [ ]:
# separando o dataset em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

Criaremos um modelo simples para servir de base.

In [ ]:
rf = RandomForestClassifier(random_state=42, bootstrap=True, oob_score=True)
rf.fit(X_train, y_train)

In [ ]:
#predisendo os valores com o modelo criado
rf_pred = rf.predict(X_test)

In [ ]:
print(classification_report(y_test, rf_pred))

Como podemos ver tivemos uma acuracia de 78% contudo um recall bem baixo nos valores positivos em virtude do desbalanceamento das classes.
Por este motivo sera interassante tanto balancearmos as classes como mudar o metodo de avalicao do modelo para um crossvalidation por exemplo para evitar que o modelo esteja aprendendo apenas uma determinada classe.

Primeiramente, vamos balancear as classes

In [ ]:
df.Churn.value_counts()

In [ ]:
# Fiz um oversampling de ate 80% da classe majoritaria, para evitaro overfiting
smt = SMOTE(sampling_strategy=0.80)
X, y = smt.fit_sample(X,y)

In [ ]:
y.value_counts()

Vamos agora, verificar qual serao os melhores parametros para ajudar o modelo utilizando o GridSearchCV

In [ ]:
# Parametros a serem testados
param_grid = {
    'n_estimators': [300, 500, 1200],
    'criterion': ('gini', 'entropy'),
    'max_features': ('log2', 'sqrt'),
    'class_weight': [{w:z} for w,z in zip([1,1.5,1],[1.5,1,1])]
}

In [ ]:
%%time
grid_rf = GridSearchCV(RandomForestClassifier(random_state=42, n_jobs=-1, bootstrap = True, oob_score = True), param_grid)
grid_rf.fit(X,y)

In [ ]:
print('\nOs melhores parametros foram: \n' + str(grid_rf.best_params_))

In [ ]:
rf = RandomForestClassifier(random_state=42, **grid_rf.best_params_)
rf.fit(X,y)

In [ ]:
cv = cross_val_score(rf, X, y, cv=10)
print(cv)
print(cv.mean())

In [ ]:
rf_pred = rf.predict(X)
print(classification_report(y, rf_pred))

In [ ]:
print(pd.crosstab(y,rf_pred, rownames=['Real'], colnames=['Predito'], margins=True))

Influencia de cada variavel na probabilidade de rotacao dos clientes.

In [ ]:
rf_feat = pd.DataFrame(rf.feature_importances_)
rf_feat['Features'] = list(df.drop(labels = ['Churn'], axis=1).columns)
rf_feat.sort_values(by=0, ascending=False).head()

In [ ]:
g = sns.barplot(0, 'Features', data= rf_feat.sort_values(by=0, ascending=False)[0:5],
               palette='Pastel1', orient='h')
g.set_xlabel('Peso')
g = g.set_title('Random Forest')

Fonte: https://www.kaggle.com/tanetboss/starter-guide-preprocessing-randomforest